In [6]:
import pandas as pd
import torch
import numpy as np
import torch.nn.functional as F
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
Data =pd.read_csv("housing.csv")
X = Data.drop(['median_house_value'] , axis=1)
Y = Data['median_house_value']
Y = Y.values.reshape(-1,1)


Place=['INLAND','ISLAND','NEAR BAY','<1H OCEAN','NEAR OCEAN']

encoder = OrdinalEncoder(
    categories=[Place],
    handle_unknown='use_encoded_value',
    unknown_value=-1
)
cat_cols = ['ocean_proximity']
new_cols = ['Place']
encoder.fit(X[cat_cols])

X[new_cols] = encoder.transform(X[cat_cols])
X = X.select_dtypes(include=['number'])

median_value = X.median(numeric_only = True)
X = X.fillna(median_value)
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
y_scaler = StandardScaler()
y_scaler.fit(Y)
Y = y_scaler.transform(Y)



X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

X_train_tensor = torch.tensor(X_train,dtype = torch.float32)
X_test_tensor = torch.tensor(X_test,dtype = torch.float32)
Y_train_tensor = torch.tensor(Y_train,dtype = torch.float32).view(-1,1)
Y_test_tensor = torch.tensor(Y_test,dtype = torch.float32).view(-1,1)
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, Y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [8]:
NUM_EPOCHS =100
RANDOM_SEED = 42
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [9]:
class Model(torch.nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.my_net = torch.nn.Sequential(
            torch.nn.Linear(in_features=num_features, out_features=6),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=6, out_features=4),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=4, out_features=1),

        )
    def forward(self, x):
        out = self.my_net(x)
        return out


torch.manual_seed(RANDOM_SEED)
model = Model(num_features=9)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

def compute_loss(net,dataloader):
    curr_loss = 0
    with torch.no_grad():
        for cnt , (features,targets) in enumerate(dataloader):
            features = features.view((-1,9)).to(DEVICE)
            targets = targets.view((-1,1)).to(DEVICE)
            out = net(features)
            loss = F.mse_loss(out, targets)
            curr_loss += loss
        return float(curr_loss)/len(dataloader)

start_time = time.time()
minibatch_cost = []
epoch_cost = []

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx , (features , targets) in enumerate(train_loader):
        features = features.view(-1,9).to(DEVICE)
        targets = targets.to(DEVICE)
        pred = model.forward(features)
        cost = F.mse_loss(pred,targets.float().view(-1,1))
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        minibatch_cost.append(cost.item())
        if not batch_idx % 64:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f'
                   %(epoch+1, NUM_EPOCHS, batch_idx,
                     len(train_loader), cost.item()))
    cost = compute_loss(model, train_loader)
    epoch_cost.append(cost)
    print('Epoch: %03d/%03d Train Cost: %.4f' % (
            epoch+1, NUM_EPOCHS, cost))
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))











Epoch: 001/100 | Batch 000/258 | Cost: 1.1468
Epoch: 001/100 | Batch 064/258 | Cost: 0.9591
Epoch: 001/100 | Batch 128/258 | Cost: 0.7613
Epoch: 001/100 | Batch 192/258 | Cost: 0.6423
Epoch: 001/100 | Batch 256/258 | Cost: 0.3078
Epoch: 001/100 Train Cost: 0.4226
Time elapsed: 0.01 min
Epoch: 002/100 | Batch 000/258 | Cost: 0.7968
Epoch: 002/100 | Batch 064/258 | Cost: 0.4765
Epoch: 002/100 | Batch 128/258 | Cost: 0.3414
Epoch: 002/100 | Batch 192/258 | Cost: 0.2433
Epoch: 002/100 | Batch 256/258 | Cost: 0.3775
Epoch: 002/100 Train Cost: 0.3733
Time elapsed: 0.01 min
Epoch: 003/100 | Batch 000/258 | Cost: 0.2889
Epoch: 003/100 | Batch 064/258 | Cost: 0.4431
Epoch: 003/100 | Batch 128/258 | Cost: 0.3208
Epoch: 003/100 | Batch 192/258 | Cost: 0.3053
Epoch: 003/100 | Batch 256/258 | Cost: 0.3923
Epoch: 003/100 Train Cost: 0.3566
Time elapsed: 0.02 min
Epoch: 004/100 | Batch 000/258 | Cost: 0.3440
Epoch: 004/100 | Batch 064/258 | Cost: 0.4065
Epoch: 004/100 | Batch 128/258 | Cost: 0.2027
E

Model(
  (my_net): Sequential(
    (0): Linear(in_features=9, out_features=6, bias=True)
    (1): ReLU()
    (2): Linear(in_features=6, out_features=4, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4, out_features=1, bias=True)
  )
)

In [11]:
model.eval()
for batch_idx , (features , targets) in enumerate(test_loader):
    features = features.view(-1,9).to(DEVICE)
    targets = targets.view(-1,1).to(DEVICE)
    pred = model.forward(features)
    cost_test = F.mse_loss(pred,targets)

print('Test Cost: %.4f' % (cost_test))


Test Cost: 0.2815


C:\Users\HP\AppData\Local\Temp\ipykernel_23508\71345490.py:8: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:837.)
  print('Test Cost: %.4f' % (cost_test))
